In [2]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

In [3]:
# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import numpy as np
from data_utils import *
from tabulate import tabulate
from naive_regresor import NaiveRegresor, NaiveRegresor_WithNoDate, NaiveRegresor_WithDateSubstraction
# from auto_ml_predictor import AutoMl_Regresor
from fill_missing_values import *
# from PCA_regresor import PCAOnlyRatingsRegresor

ModuleNotFoundError: No module named 'data_utils'

In [5]:
X,Y = get_X_Y_train()
movies_dates = get_movies_dates()
ouput_movie_date = get_output_movie_date()

In [6]:
def fill_with_none(X):
    return X
filling_nan_methods = dict()
filling_nan_methods['none'] = fill_with_none
filling_nan_methods['mean'] = fill_with_mean
filling_nan_methods['zeros'] = fill_with_zeros
filling_nan_methods['ones'] = lambda x: fill_with_num(x, 1)

In [7]:
def score_R2(y_predicted,y_actual):
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    return sqrt(mean_squared_error(y_actual, y_predicted))

In [8]:
scores = dict()
def get_min_score(k):
    return min([x[0] for x in scores[k] if isinstance(x[0], (int, float, complex))])

In [17]:
number_of_tests = 500
def check_predictor(pred, pred_name, needFill=None):
    my_scores = []
    for k,v in filling_nan_methods.items():
        if needFill is None or (k!='none') == needFill:
            cur_score = []
            for _ in range(number_of_tests):
                X_filled = v(X)
                X_train, Y_train, X_test, Y_test = random_partition(X_filled,Y)
                pred.fit(X_train, Y_train)
                Y_pred = pred.predict(X_test, Y_test[:,0])
                cur_score.append(score_R2(Y_pred, Y_test[:,1]))
            cur_score = (np.mean(cur_score), np.std(cur_score))
        else:
            cur_score = ('inf', 'inf')
        my_scores.append(cur_score)
    scores[pred_name] = my_scores 
    print("min score: ", get_min_score(pred_name), " ", pred_name)

In [18]:
check_predictor(NaiveRegresor(), 'NaiveReg', needFill=True)

min score:  0.7670156741568345   NaiveReg


In [19]:
check_predictor(NaiveRegresor_WithNoDate(), 'NaiveRegNoDate', needFill=True)

min score:  0.766795466448268   NaiveRegNoDate


In [20]:
predictor = NaiveRegresor_WithDateSubstraction(movies_dates, ouput_movie_date)
check_predictor(predictor, 'NaiveRegDateSub', needFill=True)

min score:  0.7639765862283672   NaiveRegDateSub


In [21]:
# check_predictor(AutoMl_Regresor(), 'AutoMl_Regresor')

In [22]:
# regresors = dict()
# from sklearn.linear_model import ARDRegression,BayesianRidge,ElasticNet,ElasticNetCV,HuberRegressor,LinearRegression,LogisticRegression,LogisticRegressionCV,PassiveAggressiveRegressor,RandomizedLogisticRegression,Ridge,TheilSenRegressor
# # regresors['ARDRegression'] = ARDRegression()
# regresors['BayesianRidge'] = BayesianRidge()
# regresors['ElasticNet'] = ElasticNet()
# regresors['ElasticNetCV'] = ElasticNetCV() 
# regresors['HuberRegressor'] = HuberRegressor()
# regresors['LinearRegression'] = LinearRegression()
# regresors['LogisticRegression'] = LogisticRegression()
# regresors['LogisticRegressionCV'] = LogisticRegressionCV()
# regresors['PassiveAggressiveRegressor'] = PassiveAggressiveRegressor()
# regresors['Ridge'] = Ridge()
# regresors['TheilSenRegressor'] = TheilSenRegressor()


In [23]:
# number_of_component = [1,2,4,8,10,16,24,32]

# for k,v in regresors.items():
#     for noc in number_of_component:
#         check_predictor(PCAOnlyRatingsRegresor(noc, v), 'PCAOnlyRatingsRegresor_%s_%d'%(k,noc), True)

In [24]:
headers = []
for h in filling_nan_methods.keys():
    headers.append(h + "_mean")
    headers.append(h + "_std")
values = [[k] + list(np.array(scores[k]).flatten()) for k in sorted(scores.keys(), key=get_min_score)]
print(tabulate(values, headers=headers, tablefmt='orgtbl', floatfmt=".4f"))

|                 |   none_mean |   none_std |   mean_mean |   mean_std |   zeros_mean |   zeros_std |   ones_mean |   ones_std |
|-----------------+-------------+------------+-------------+------------+--------------+-------------+-------------+------------|
| NaiveRegDateSub |         inf |        inf |      0.7772 |     0.0096 |       0.7656 |      0.0091 |      0.7640 |     0.0086 |
| NaiveRegNoDate  |         inf |        inf |      0.7723 |     0.0093 |       0.7747 |      0.0083 |      0.7668 |     0.0100 |
| NaiveReg        |         inf |        inf |      0.7769 |     0.0093 |       0.7678 |      0.0088 |      0.7670 |     0.0085 |
